<a href="https://www.kaggle.com/code/aruneembhowmick/keras-cnns-for-tmnist-numeral-classification?scriptVersionId=210511935" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [ ]:
tmnist_data = pd.read_csv('/kaggle/input/tmnist-typeface-mnist/TMNIST_Data.csv')
tmnist_data

In [ ]:
pixels = tmnist_data.drop(columns = ['names', 'labels']).values / 255.0
pixels = pixels.reshape(pixels.shape[0], 28, 28, 1)

label_encoder = LabelEncoder()
numerals = label_encoder.fit_transform(tmnist_data['labels'])
numerals = to_categorical(numerals)

pixels_train, pixels_test, numerals_train, numerals_test = train_test_split(pixels, numerals, test_size = 0.2, random_state = 42)

In [ ]:
def visualize_image_grid(images, grid_size = (10, 10)):
    
    rows, cols = grid_size
    plt.figure(figsize = (cols * 2, rows * 2))

    for i in range(rows * cols):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(images[i].squeeze(), cmap = 'gray')
        plt.axis("off")

    plt.tight_layout()
    plt.show()

visualize_image_grid(pixels_train[:100], grid_size = (10, 10))

In [ ]:
model = models.Sequential([
    Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation = 'relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(10, activation = 'softmax')
])

checkpoint = ModelCheckpoint(
    filepath = 'tmnist_numeral_prediction_model.keras',   
    monitor = 'val_accuracy',     
    save_best_only = True,        
    mode = 'max',                 
    verbose = 1                   
)

model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

model.summary()

In [ ]:
history = model.fit(pixels_train, numerals_train, 
                    epochs = 25, 
                    validation_data = (pixels_test, numerals_test),
                    callbacks = [checkpoint])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('TMNIST Numeral Prediction CNN Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('TMNIST Numeral Prediction CNN Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc = 'upper right')
plt.show()

In [ ]:
_, test_acc = model.evaluate(pixels_test, numerals_test, verbose = 0)
print(f'Test Accuracy: {100 * test_acc:.2f}%')

_, train_acc = model.evaluate(pixels_train, numerals_train, verbose = 0)
print(f'Train Accuracy: {100 * train_acc:.2f}%')

_, acc = model.evaluate(pixels, numerals, verbose = 0)
print(f'General Accuracy: {100 * acc:.2f}%')

In [ ]:
predicted_probs = model.predict(pixels_test)
predicted_labels = np.argmax(predicted_probs, axis = 1)
true_labels = np.argmax(numerals_test, axis = 1)
correct_indices = np.where(predicted_labels == true_labels)[0]

def display_correct_predictions(images, true_labels, predictions, indices, n_display = 10, class_names = None):

    if len(indices) > 0:
        display_indices = np.random.choice(indices, size = min(n_display, len(indices)), replace = False)

        plt.figure(figsize = (1.5 * n_display, 3 * n_display))
        for i, idx in enumerate(display_indices, start=1):
            img = images[idx].squeeze()
            pred_label = predictions[idx]
            true_label = true_labels[idx]
            plt.subplot(1, n_display, i)
            plt.imshow(img, cmap = 'gray')
            title_text = f'Predicted: {class_names[pred_label] if class_names else pred_label}\nTrue: {class_names[true_label] if class_names else true_label}'
            plt.title(title_text)
            plt.axis('off')
        plt.show()
    else:
        print('No correct predictions to display.')

display_correct_predictions(pixels_test, true_labels, predicted_labels, correct_indices)

In [ ]:
numerals_pred = model.predict(pixels_test)
numerals_pred_classes = np.argmax(numerals_pred, axis = 1)
numerals_true = np.argmax(numerals_test, axis=1)

conf_matrix = confusion_matrix(numerals_true, numerals_pred_classes)

plt.figure(figsize = (10, 10))
sns.heatmap(conf_matrix, annot = True, fmt = 'd', cmap = 'gray')
plt.title('Confusion Matrix')
plt.ylabel('True Numeral')
plt.xlabel('Predicted Numeral')
plt.yticks(rotation = 0)
plt.show()